# Gallery

In [ ]:
import base64, json, jinja2, pandas as pd, IPython.display as D
from pathlib import Path
from matplotlib import cm
HERE = Path.cwd()
ATEST = HERE.parent / "atest/output"
IMG = sorted([
    p for p in sorted(ATEST.rglob("*.png"))
    if "pabot" not in str(p)
    and "notebook-index" in str(p)
    and "linux_38" in str(p)
])
CELLS = {str(i).split('/screens/')[1]: D.Image(i) for i in IMG if "cell" in str(i)}
MROS = {
    j.name: json.loads(j.read_text(encoding="utf-8")) for j in (HERE.parent / "build/nbwidgets").glob("*.json")
}

In [ ]:
rows = []
for path, data in MROS.items():
    for mro in data["mro_counts"]:
        rows += [dict(Example=path.replace(".json", ""), mro=mro["mro"], count=mro["count"])]
df = pd.DataFrame(rows)
df["fqn"] = df["mro"].apply(lambda x: x[0])
df["cls"] = df["fqn"].apply(lambda x: x.rsplit(".")[-1])
df["mod"] = df["fqn"].apply(lambda x: x.rsplit(".", 1)[0])
df["wxyz"] = df["mod"].str.startswith("wxyz")

In [ ]:
CAROUSEL_TMPL = jinja2.Template("""
<div id="carousel-wxyz-{{ label }}" class="carousel slide" data-ride="carousel">
  <ol class="carousel-indicators">
    {% for img in imgs %}
    <li data-target="#carousel-wxyz-{{ label }}" data-slide-to="{{ loop.index0 }}"></li>
    {% endfor %}
  </ol>
  <div class="carousel-inner">
    {% for img in imgs %}
        <div class="carousel-item{% if not loop.index0 %} active{% endif %}">
          <img class="d-block w-100" src="{{ img }}">
        </div>
    {% endfor %}
  </div>
  <a class="carousel-control-prev" href="#carousel-wxyz-{{ label }}" role="button" data-slide="prev">
    <span class="carousel-control-prev-icon" aria-hidden="true"></span>
    <span class="sr-only">Previous</span>
  </a>
  <a class="carousel-control-next" href="#carousel-wxyz-{{ label }}" role="button" data-slide="next">
    <span class="carousel-control-next-icon" aria-hidden="true"></span>
    <span class="sr-only">Next</span>
  </a>
</div>
""")

In [ ]:
def png_data_uri(path):
    return 'data:image/png;base64,{}'.format(base64.b64encode(path.read_bytes()).decode("utf-8"))

## Screenshots
These are images generated during the [Robot Framework](https://robotframework.org) tests.

In [ ]:
outputs = []
for parent in sorted(set([p.parent for p in IMG])):
    context = dict(
        label=parent.name,
        imgs=[png_data_uri(i) for i in IMG if parent == i.parent]
    )
    outputs += [D.HTML(CAROUSEL_TMPL.render(**context))]
D.display(*outputs);

## WXYZ in Notebooks

In [ ]:
piv = pd.pivot_table(df[df["wxyz"]], values="count", index=['Example'], columns=['fqn']).T.reset_index().fillna(0)
piv["module"] = piv["fqn"].apply(lambda x: ".".join(x.rsplit(".", 1)[0:1]))
piv["widget"] = piv["fqn"].apply(lambda x: x.rsplit(".", 1)[1])
piv_out = piv[[c for c in piv.columns if c not in ["fqn"]]].set_index(["module", "widget"]).astype("int")
st = piv_out.style
st.background_gradient("gray_r").set_properties(**{"text-align": "center", "min-width": "4rem"})